# PINN Evaluation

In [ ]:
import os
from dotenv import load_dotenv
import sys

In [ ]:
load_dotenv()

lib_dir = os.environ.get("LIB_DIR")
sys.path.append(lib_dir)

data_dir = os.environ.get("DATA_DIR")
model_dir = os.path.join(data_dir, "models")
points_file_name = "points_final.csv"
measurements_file_name = "measurements.csv"

In [ ]:
import pandas as pd
import torch
import yaml
from pinn import PINN

In [ ]:
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
wing_df = pd.read_csv(os.path.join(data_dir, points_file_name))
measurements_df = pd.read_csv(os.path.join(data_dir, measurements_file_name))

device = torch.device("cpu")

x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

xyz_range = (overall_min, overall_max)

In [ ]:
# load model
model_name = config["model_name"]
model_checkpoint_num = config["model_checkpoint_num"]

pinn = PINN.load_from_checkpoint_for_testing(model_dir, model_name, model_checkpoint_num).to(device)

In [ ]:
# eval domain
x_max = config["x_max"]
y_max = config["y_max"]
z_max = config["z_max"]
# t_max = 2 # TODO: set t>2 to test outside of training domain

In [ ]:
pinn.eval_pinn(s_df=wing_df, u_df=measurements_df, 
               Nf=10000, N0=10000, Nb=10000, Ns=10000, Nu=10,
               x_max=x_max, y_max=y_max, z_max=z_max,
               in_velocity=1, mu=1, rho=1,
               device=device)

In [ ]:
pinn.print_current_metrics()

In [ ]:
pinn.print_all_metrics()